In [1]:
from autoencoder import *

In [2]:
theta = np.linspace(-np.pi,0.0*np.pi,500, dtype=np.float32)
data = np.vstack([np.cos(theta), np.sin(theta)]).T
dataset = tf.data.Dataset.from_tensor_slices(data)

In [3]:
def make_datastream(dataset, batchsize=100, buffer_size=1000):
    repeat_dataset = dataset.repeat()
    shuffled_dataset = repeat_dataset.shuffle(buffer_size=buffer_size)
    batched_dataset = shuffled_dataset.batch(batchsize)
    iterator = batched_dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    return tf.stack(next_element)

In [5]:
graph = tf.Graph()
with graph.as_default():
    tr_x = make_datastream(dataset)
    au = PolyAutoencoder(2,1,tr_x, 5,5)
    init=tf.global_variables_initializer()

In [6]:
try:
    sess.close()
except:
    pass
sess = tf.InteractiveSession(graph=graph)
init.run(session=sess)

In [7]:
for i in xrange(10000):
    #train_step.run(session=sess)
    sess.run(au.train_step)
    #if i%1000==99:
    #    print au.goal.eval(session=sess)